In [1]:
#Tensorflow version

import tensorflow as tf
print(tf.__version__)

2.9.1


In [2]:
#Importing libraries

import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Conv2D, Dense, Flatten, BatchNormalization, Dropout, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model
import matplotlib.pyplot as plt
from keras.applications.efficientnet import EfficientNetB0
from keras.preprocessing import image
#from keras.preprocessing.image import load_img
from keras.models import Sequential
import numpy as np
from tensorflow.keras import regularizers

In [3]:
#Datasets path

# 3 percent with HYP 10 percent
train_dir='/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/3 percent with HYP 10 percent - v2/Training/'
val_dir='/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/3 percent with HYP 10 percent - v2/Validation/'


In [4]:
from tensorflow.keras import layers

train_data = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                 image_size=(256,256),
                                                                 label_mode='categorical', batch_size=128, 
                                                                 )
val_data = tf.keras.preprocessing.image_dataset_from_directory(directory=val_dir,
                                                                 image_size=(256,256),
                                                                 label_mode='categorical', batch_size=128,
                                                                )

Found 30246 files belonging to 5 classes.


2023-01-17 08:09:56.979606: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-17 08:09:57.004103: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-17 08:09:57.004332: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-17 08:09:57.018216: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Found 8051 files belonging to 5 classes.


In [5]:
#Normalization of the data

nrmzln_layer = layers.experimental.preprocessing.Rescaling(1./255)

train_data = train_data.map(lambda x,
                          y: (nrmzln_layer(x), y))

val_data = val_data.map(lambda x,
                          y: (nrmzln_layer(x), y))

In [6]:
# Instantiate base_model with pre-trained weights

#import tensorflow.python.keras.applications.VGG16

base_model= tf.keras.applications.VGG19(weights= 'imagenet', input_shape=(256,256, 3), include_top=False) 

In [7]:
# Freezing the base_model for transfer learning
base_model.trainable= False

In [8]:
# Creating a new_model on the top
inputs= tf.keras.Input(shape= (256, 256, 3), name="input_layer")

In [9]:
x= base_model(inputs,training= False)

In [10]:
# Average pool the outputs of the base model
x = tf.keras.layers.GlobalAveragePooling2D(name='Global_Average_Pooling_Layer')(x)

In [11]:
# Output layer
u = tf.keras.layers.Dense(1050, activation="relu")(x)
u = tf.keras.layers.Dense(512, activation="relu")(u)
u = tf.keras.layers.Dense(256, activation="relu")(u)
u = tf.keras.layers.Dense(128, activation="relu")(u)
outputs = tf.keras.layers.Dense(5, activation="softmax",name="output_layer")(u)

In [12]:
# Combine the model
model = tf.keras.Model(inputs, outputs) 

In [13]:
#printing model summary

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 256, 256, 3)]     0         
                                                                 
 vgg19 (Functional)          (None, 8, 8, 512)         20024384  
                                                                 
 Global_Average_Pooling_Laye  (None, 512)              0         
 r (GlobalAveragePooling2D)                                      
                                                                 
 dense (Dense)               (None, 1050)              538650    
                                                                 
 dense_1 (Dense)             (None, 512)               538112    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                             

In [14]:
# Compile the model

from tensorflow.keras.metrics import SensitivityAtSpecificity,SpecificityAtSensitivity,Recall,Precision

model.compile(loss= tf.losses.categorical_crossentropy, 
      optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy',SensitivityAtSpecificity(0.5),SpecificityAtSensitivity(0.5),Recall(0.5),Precision(0.5)])

In [15]:
#callback

class myCallback(tf.keras.callbacks.Callback):
     def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.99): 
          print("\nReached 99% accuracy so cancelling training!")
          self.model.stop_training = True

callbacks = myCallback()

In [16]:
# Fit the model
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("ECG_Spectrogram_Model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
history = model.fit(train_data, steps_per_epoch=len(train_data), epochs=50, validation_data=val_data, validation_steps=len(val_data),callbacks = [callbacks,checkpoint])

Epoch 1/50


2023-01-17 08:10:22.049171: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-01-17 08:10:22.421330: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


237/237 [==============================] - ETA: 0s - loss: 1.3174 - accuracy: 0.4248 - sensitivity_at_specificity: 0.8383 - specificity_at_sensitivity: 0.8185 - recall: 0.1660 - precision: 0.6710
Epoch 1: val_accuracy improved from -inf to 0.53074, saving model to ECG_Spectrogram_Model.h5
237/237 [==============================] - 227s 908ms/step - loss: 1.3174 - accuracy: 0.4248 - sensitivity_at_specificity: 0.8383 - specificity_at_sensitivity: 0.8185 - recall: 0.1660 - precision: 0.6710 - val_loss: 1.1247 - val_accuracy: 0.5307 - val_sensitivity_at_specificity: 0.9201 - val_specificity_at_sensitivity: 0.9007 - val_recall: 0.3663 - val_precision: 0.6423
Epoch 2/50
237/237 [==============================] - ETA: 0s - loss: 0.9872 - accuracy: 0.6018 - sensitivity_at_specificity: 0.9548 - specificity_at_sensitivity: 0.9411 - recall: 0.4473 - precision: 0.7183
Epoch 2: val_accuracy improved from 0.53074 to 0.59086, saving model to ECG_Spectrogram_Model.h5
237/237 [========================

Epoch 14/50
237/237 [==============================] - ETA: 0s - loss: 0.6709 - accuracy: 0.7445 - sensitivity_at_specificity: 0.9905 - specificity_at_sensitivity: 0.9859 - recall: 0.6791 - precision: 0.8023
Epoch 14: val_accuracy did not improve from 0.68203
237/237 [==============================] - 200s 843ms/step - loss: 0.6709 - accuracy: 0.7445 - sensitivity_at_specificity: 0.9905 - specificity_at_sensitivity: 0.9859 - recall: 0.6791 - precision: 0.8023 - val_loss: 0.8840 - val_accuracy: 0.6726 - val_sensitivity_at_specificity: 0.9627 - val_specificity_at_sensitivity: 0.9700 - val_recall: 0.6176 - val_precision: 0.7263
Epoch 15/50
237/237 [==============================] - ETA: 0s - loss: 0.6472 - accuracy: 0.7527 - sensitivity_at_specificity: 0.9909 - specificity_at_sensitivity: 0.9872 - recall: 0.6912 - precision: 0.8057
Epoch 15: val_accuracy improved from 0.68203 to 0.69085, saving model to ECG_Spectrogram_Model.h5
237/237 [==============================] - 201s 848ms/step - 

Epoch 27/50
237/237 [==============================] - ETA: 0s - loss: 0.5502 - accuracy: 0.7897 - sensitivity_at_specificity: 0.9966 - specificity_at_sensitivity: 0.9932 - recall: 0.7422 - precision: 0.8346
Epoch 27: val_accuracy did not improve from 0.69917
237/237 [==============================] - 200s 842ms/step - loss: 0.5502 - accuracy: 0.7897 - sensitivity_at_specificity: 0.9966 - specificity_at_sensitivity: 0.9932 - recall: 0.7422 - precision: 0.8346 - val_loss: 1.0156 - val_accuracy: 0.6658 - val_sensitivity_at_specificity: 0.9422 - val_specificity_at_sensitivity: 0.9615 - val_recall: 0.6147 - val_precision: 0.7037
Epoch 28/50
237/237 [==============================] - ETA: 0s - loss: 0.5457 - accuracy: 0.7913 - sensitivity_at_specificity: 0.9961 - specificity_at_sensitivity: 0.9932 - recall: 0.7432 - precision: 0.8340
Epoch 28: val_accuracy did not improve from 0.69917
237/237 [==============================] - 240s 1s/step - loss: 0.5457 - accuracy: 0.7913 - sensitivity_at_

Epoch 40/50
237/237 [==============================] - ETA: 0s - loss: 0.4539 - accuracy: 0.8273 - sensitivity_at_specificity: 0.9986 - specificity_at_sensitivity: 0.9967 - recall: 0.7935 - precision: 0.8596
Epoch 40: val_accuracy did not improve from 0.69917
237/237 [==============================] - 200s 842ms/step - loss: 0.4539 - accuracy: 0.8273 - sensitivity_at_specificity: 0.9986 - specificity_at_sensitivity: 0.9967 - recall: 0.7935 - precision: 0.8596 - val_loss: 1.6164 - val_accuracy: 0.6204 - val_sensitivity_at_specificity: 0.8803 - val_specificity_at_sensitivity: 0.9470 - val_recall: 0.5932 - val_precision: 0.6428
Epoch 41/50
237/237 [==============================] - ETA: 0s - loss: 0.4547 - accuracy: 0.8258 - sensitivity_at_specificity: 0.9985 - specificity_at_sensitivity: 0.9965 - recall: 0.7921 - precision: 0.8617
Epoch 41: val_accuracy did not improve from 0.69917
237/237 [==============================] - 240s 1s/step - loss: 0.4547 - accuracy: 0.8258 - sensitivity_at_

In [17]:
#Acquiring Performance metrics
################################################## for VGG-19 Model ############################################

Training_Accuracy=history.history['accuracy']
Validation_Accuracy=history.history['val_accuracy']
Validation_Specificity=history.history['val_specificity_at_sensitivity']
Validation_Sensitivity=history.history['val_sensitivity_at_specificity']
Validation_Recall=history.history['val_recall']
Validation_Precision=history.history['val_precision']
Validation_Loss=history.history['val_loss']

print("Training Accuracy: ",max(Training_Accuracy))
print("Validation Accuracy: ",max(Validation_Accuracy))
print("Validation Specificity: ",max(Validation_Specificity))
print("Validation Sensitivity: ",max(Validation_Sensitivity))
print("Validation Recall: ",max(Validation_Recall))
print("Validation Precision: ",max(Validation_Precision))
print("Validation Loss: ",min(Validation_Loss))

Training Accuracy:  0.8554519414901733
Validation Accuracy:  0.699167788028717
Validation Specificity:  0.9748478531837463
Validation Sensitivity:  0.9762762188911438
Validation Recall:  0.6528381705284119
Validation Precision:  0.7605445981025696
Validation Loss:  0.8098897933959961


In [17]:
#Acquiring Performance metrics
################################################## for DenseNet-201 Model ############################################

Training_Accuracy=history.history['accuracy']
Validation_Accuracy=history.history['val_accuracy']
Validation_Specificity=history.history['val_specificity_at_sensitivity']
Validation_Sensitivity=history.history['val_sensitivity_at_specificity']
Validation_Recall=history.history['val_recall']
Validation_Precision=history.history['val_precision']
Validation_Loss=history.history['val_loss']

print("Training Accuracy: ",max(Training_Accuracy))
print("Validation Accuracy: ",max(Validation_Accuracy))
print("Validation Specificity: ",max(Validation_Specificity))
print("Validation Sensitivity: ",max(Validation_Sensitivity))
print("Validation Recall: ",max(Validation_Recall))
print("Validation Precision: ",max(Validation_Precision))
print("Validation Loss: ",min(Validation_Loss))

Training Accuracy:  0.9713350534439087
Validation Accuracy:  0.7312135100364685
Validation Specificity:  0.9834182262420654
Validation Sensitivity:  0.980126678943634
Validation Recall:  0.6940752863883972
Validation Precision:  0.7783111929893494
Validation Loss:  0.7456523776054382


In [17]:
#Acquiring Performance metrics
################################################## for EfficientNet-0 Model ############################################

Training_Accuracy=history.history['accuracy']
Validation_Accuracy=history.history['val_accuracy']
Validation_Specificity=history.history['val_specificity_at_sensitivity']
Validation_Sensitivity=history.history['val_sensitivity_at_specificity']
Validation_Recall=history.history['val_recall']
Validation_Precision=history.history['val_precision']
Validation_Loss=history.history['val_loss']

print("Training Accuracy: ",max(Training_Accuracy))
print("Validation Accuracy: ",max(Validation_Accuracy))
print("Validation Specificity: ",max(Validation_Specificity))
print("Validation Sensitivity: ",max(Validation_Sensitivity))
print("Validation Recall: ",max(Validation_Recall))
print("Validation Precision: ",max(Validation_Precision))
print("Validation Loss: ",min(Validation_Loss))

Training Accuracy:  0.21887192130088806
Validation Accuracy:  0.24394485354423523
Validation Specificity:  0.4127126932144165
Validation Sensitivity:  0.4759657084941864
Validation Recall:  0.0
Validation Precision:  0.0
Validation Loss:  1.6004568338394165


In [17]:
#Acquiring Performance metrics
################################################## for ResNet-50 Model ############################################

Training_Accuracy=history.history['accuracy']
Validation_Accuracy=history.history['val_accuracy']
Validation_Specificity=history.history['val_specificity_at_sensitivity']
Validation_Sensitivity=history.history['val_sensitivity_at_specificity']
Validation_Recall=history.history['val_recall']
Validation_Precision=history.history['val_precision']
Validation_Loss=history.history['val_loss']

print("Training Accuracy: ",max(Training_Accuracy))
print("Validation Accuracy: ",max(Validation_Accuracy))
print("Validation Specificity: ",max(Validation_Specificity))
print("Validation Sensitivity: ",max(Validation_Sensitivity))
print("Validation Recall: ",max(Validation_Recall))
print("Validation Precision: ",max(Validation_Precision))
print("Validation Loss: ",min(Validation_Loss))

Training Accuracy:  0.7327911257743835
Validation Accuracy:  0.612594723701477
Validation Specificity:  0.9434542059898376
Validation Sensitivity:  0.9520556330680847
Validation Recall:  0.5203080177307129
Validation Precision:  0.7141714692115784
Validation Loss:  1.0098254680633545
